In [1]:
# import required libraries
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import requests
import pandas as pd
import lxml.html as lh

## 1. Get Product Page URL

Using Web Scraping, I will obtain the URLs of each product page where its nutrition information are to be collected. I will collect the URLs of products shown on each category's main page.

category: 'Meat & Fish' https://groceries.morrisons.com/browse/meat-fish-179549

In [4]:
link='https://groceries.morrisons.com/browse/meat-fish-179549'
options = Options()
options.binary_location = "/Applications/Chrome 2.app/Contents/MacOS/Google Chrome"
driver = webdriver.Chrome(options = options, executable_path='/Users/haneul/Downloads/chromedriver')
driver.get(link+'?display=500')

#Scrolling the page every 2 second to the end of the page
last_height = driver.execute_script("return document.body.scrollHeight")
h=0
while h<last_height:
    h += 450
    time.sleep(2)
    driver.execute_script(f"window.scrollTo(0, {h});")
    print('\r', "Wait... Parsing", int(h/last_height*100), "%" , end='')


 Wait... Parsing 100 %

In [5]:
# find the source code of item list.
html = driver.page_source
soup=BeautifulSoup(html, 'lxml')
list_source=soup.find('div', class_='main-column')

In [6]:
fp = open('mf-soup.txt','w')
fp.write(soup.prettify())
fp.close()

In [7]:
oos_url_list=[]
content=list_source.find_all('div', {'class':"fop-contentWrapper"})
for c in content:
    if c.find('span',class_='fop-mark-oos fop-image-corner', text='Out of Stock'):
        url_tail=c.a['href']
        url='https://groceries.morrisons.com'+url_tail
        oos_url_list.append(url)

In [8]:
url_list=[]
# grab all the urls using for loop
for url_source in list_source.find_all('div', class_='fop-contentWrapper'):
    url_tail=url_source.a.attrs['href']
    url='https://groceries.morrisons.com'+url_tail
    url_list.append(url)

In [9]:
for ele in oos_url_list:
    url_list.remove(ele)

In [10]:
url_list

['https://groceries.morrisons.com//products/morrisons-market-st-salmon-fillets-403510011',
 'https://groceries.morrisons.com//products/morrisons-the-best-pork-chipolatas-12-pack-121932011',
 'https://groceries.morrisons.com//products/morrisons-market-st-british-sirloin-steak-408297011',
 'https://groceries.morrisons.com//products/morrisons-chicken-thigh-fillets-boneless-252637011',
 'https://groceries.morrisons.com//products/morrisons-the-best-thick-pork-sausages-121931011',
 'https://groceries.morrisons.com//products/morrisons-2-garlic-breaded-chicken-kievs-116605011',
 'https://groceries.morrisons.com//products/morrisons-spring-lamb-leg-roast-fillet-297704011',
 'https://groceries.morrisons.com//products/morrisons-breaded-chicken-goujons-116603011',
 'https://groceries.morrisons.com//products/morrisons-the-best-hampshire-breed-dry-cured-smoked-back-bacon-386033011',
 'https://groceries.morrisons.com//products/morrisons-the-best-6-hampshire-unsmoked-cured-back-bacon-rashers-212572011'

In [11]:
len(url_list)

402

In [12]:
# remove duplicates
url_list=list(dict.fromkeys(url_list))

In [13]:
len(url_list)

402

## Creating For Loop for all-product pages webscraping

In [14]:
names=[]
for i in range(1, len(url_list)+1):
    text='item'
    name=text+str(i)
    names.append(name)
len(names)


402

In [15]:
file_name='meat_fish.csv'
url_df=pd.DataFrame({'file_name.split(.)[0]': url_list})
url_df.to_csv('url_'+file_name, index=False)

In [51]:
df=pd.read_csv('url_meat_fish.csv')
url_list=df['url_list'].tolist()

In [52]:
url_list[0]

'https://groceries.morrisons.com/products/morrisons-market-st-salmon-fillets-403510011'

In [109]:
# in each item page, find the source code where each item's nutrition information is written
appended_data=[]
for I in range(0, len(url_list)):
    # get source code from website using 'requests' library
    item_source=requests.get(url_list[I])

    # create BeautifulSoup object
    item_doc=lh.fromstring(item_source.content)
    
    # parse data that are stored between <tr>..</tr> of HTML
    tr_elements = item_doc.xpath('//tr')
    
    if len(tr_elements)==0:
        pass  # go to I+=1 skipping else statement
    else:
        # create the empty list
        col=[]

        # for each row, store each first element(header) and empty list
        for t in tr_elements[0]:
            type=t.text_content()
            col.append((type,[]))

        # store the data on the second row onwards
        for j in range(1, len(tr_elements)):  #iterate from 1st row to th last one.
            T=tr_elements[j]

            # set the i as index of the column
            i=0

            # iterate through each element of the row
            for t in T.iterchildren():
                data=t.text_content()

                # append the data to the empty list of i'the column
                col[i][1].append(data)

                # increment i for the next column.
                i+=1
        
        # obtain the item name
        item_name_elements = item_doc.xpath('//header[@class="bop-title"]')
        item_name=item_name_elements[0][0].text_content()

        # add the item name on the list
        col.append(('item name',[]))
        for i in range(0,len(col[0][1])):
            col[-1][1].append(item_name)
        
        # obtain the price
        price_elements = item_doc.xpath('//*[@id="overview"]/section[2]/div[1]/div/h2')
        price=price_elements[0].text_content()

        # add the price on the list
        col.append(('price',[]))
        for i in range(0,len(col[0][1])):
            col[-1][1].append(price)
        
        # obtain the image url
        image_elements = item_doc.xpath('//*[@id="overview"]/section[1]/div/div/div/img/@src')
        image_url=image_elements[0]

        # add the image url on the list
        col.append(('image_url',[]))
        for i in range(0,len(col[0][1])):
            col[-1][1].append(image_url)
              
        # add the product page url on the list
        col.append(('product_url',[]))
        for i in range(0,len(col[0][1])):
            col[-1][1].append(url_list[I])
        
        # obtain the ingredients
        ingredient_elements = item_doc.xpath('//*[@id="productInformation"]/div[3]/div[2]/div[2]/div/div/div')
        ingredient=ingredient_elements[0].text_content()

        # add the ingredients on the list
        col.append(('ingredients',[]))
        for i in range(0,len(col[0][1])):
            col[-1][1].append(ingredient)
       
        # create a dictionary with the list, 'col'
        Dict={title:column for (title,column) in col}
        
        # convert a dictionary to a dataframe
        df=pd.DataFrame(Dict)
        
        # store the dataframe in list
        appended_data.append(df)
        
       
        # create a dictionary with the list, 'col'
        Dict={title:column for (title,column) in col}
        
        # convert a dictionary to a dataframe
        df=pd.DataFrame(Dict)
        
        
    I+=1


In [108]:
appended_data=pd.concat(appended_data)
appended_data.to_csv('meat_fish_update.csv', index=False)

ValueError: No objects to concatenate

In [20]:
# In case that you wish to save each dataframes as csv with item_name
# for group, data in df.groupby(item_name):;data.to_csv(f"{group}.csv",index=False)